## Computer Vision Task 9

### Libraries Used

In [ ]:
import cv2
import numpy as np

### Apply a code to estimate the motion in video using Lucas-Kanade method

In [ ]:
# The video read is read in as
# a VideoCapture object
cap = cv2.VideoCapture("1.mp4") # video path

# params for corner detection
featureParams = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# paramaters for lucas kanade optical flow
lkParams = dict(winSize = (15,15), maxLevel = 2)

# creating some random colors
color = np.random.randint(0,255,(100,3))

# take the first frame and corners in it
ret, old_frame = cap.read()
oldGray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

p0 = cv2.goodFeaturesToTrack(oldGray, mask=None, **featureParams)

# creating a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    frameGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1,st,err = cv2.calcOpticalFlowPyrLK(oldGray, frameGray, p0, None, **lkParams)
    
    # select good points
    goodNew = p1[st == 1]
    goodOld = p0[st == 1]
    
    # draw the tracks
    for i, (new, old) in enumerate(zip(goodNew, goodOld)):
        
        a, b = new.ravel()
        c, d = old.ravel()
        
       # mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), -1)
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)

        img = cv2.add(frame, mask)
        cv2.imshow("Video", img)
        
        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        # updating previous frame and points
        oldGray = frameGray.copy()
        p0 = goodNew.reshape(-1, 1, 2)
        
cv2.destroyAllWindows()
cap.release()

### Apply a code to estimate the motion in video using Lucas-Kanade method (Modified)

In [2]:
# Function to estimate motion using Lucas-Kanade method
def estimate_motion(video_path):
    # Read the video file
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_points = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    
    # Set parameters for Lucas-Kanade optical flow
    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    # Create some random colors for visualizing motion tracks
    color = np.random.randint(0, 255, (100, 3))
    
    while True:
        # Read the next frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate optical flow using Lucas-Kanade method
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)
        
        # Select good points
        good_prev = prev_points[status == 1]
        good_next = next_points[status == 1]
        
        # Draw motion tracks
        for i, (prev_pt, next_pt) in enumerate(zip(good_prev, good_next)):
            x1, y1 = prev_pt.ravel().astype(int)
            x2, y2 = next_pt.ravel().astype(int)
            cv2.line(frame, (x1, y1), (x2, y2), color[i].tolist(), 2)
            cv2.circle(frame, (x2, y2), 5, color[i].tolist(), -1)
        
        # Display the resulting frame
        cv2.imshow('Video', frame)
        
        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Update previous frame and points
        prev_gray = gray.copy()
        prev_points = good_next.reshape(-1, 1, 2)
    
    # Release the video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Path to the input video
video_path = '1.mp4' # video path

# Call the function to estimate motion
estimate_motion(video_path)

### End